In [3]:
# reference: https://huggingface.co/docs/diffusers/main/en/api/pipelines/stable_unclip#text-guided-image-to-image-variation

# NEED THIS TO SAVE TO BIGGER DRIVE
import os
# os.environ['HF_HOME'] = '/datastor1/jiahuikchen/hf_cache' # CS A40 box
os.environ['HF_HOME'] = '/home/jc98685/hf_cache' # MIDI-02

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from diffusers import StableUnCLIPImg2ImgPipeline
from diffusers.utils import load_image, logging
import torch

import torchvision.transforms as transforms
import torchvision.datasets as datasets

### Image and Text Conditioned UnClip

In [2]:
img_txt_pipe = StableUnCLIPImg2ImgPipeline.from_pretrained(
    "stabilityai/stable-diffusion-2-1-unclip", torch_dtype=torch.float16, 
)
img_txt_pipe.set_progress_bar_config(disable=True)

img_txt_pipe = img_txt_pipe.to("cuda:0")

Loading pipeline components...:   0%|          | 0/9 [00:00<?, ?it/s]

In [ ]:
# text and image generation
# prompts_and_imgs is list of tuples (prompt, image)
def gen_images_from_text_and_img(prompts_and_imgs):
    for tup in prompts_and_imgs:
        prompt = tup[0]
        img = tup[1]
        images = img_txt_pipe(img, prompt=prompt, num_images_per_prompt=4).images

        f, axarr = plt.subplots(1,4, figsize=(20, 20))
        axarr[0].imshow(images[0]); axarr[0].axis("off"); axarr[0].set_title(prompt)
        axarr[1].imshow(images[1]); axarr[1].axis("off"); axarr[1].set_title(prompt)
        axarr[2].imshow(images[2]); axarr[2].axis("off"); axarr[2].set_title(prompt)
        axarr[3].imshow(images[3]); axarr[3].axis("off"); axarr[3].set_title(prompt)

## Mixup

Original paper repo:
https://github.com/facebookresearch/mixup-cifar10  

Most of below is copied from: https://github.com/facebookresearch/mixup-cifar10/blob/main/train.py#L152

In [ ]:
transform_train = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465),
                             (0.2023, 0.1994, 0.2010)),
    ])

trainset = datasets.CIFAR10(root='/home/jc98685/hf_cache/data', train=True, download=True,
                            transform=transform_train)

trainloader = torch.utils.data.DataLoader(trainset,
                                          batch_size=args.batch_size,
                                          shuffle=True, num_workers=8)

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

# for batch_idx, (inputs, targets) in enumerate(trainloader):
batch_idx, (inputs, targets) = list(enumerate(trainloader))[0]
if use_cuda:
    inputs, targets = inputs.cuda(), targets.cuda()

inputs, targets_a, targets_b, lam = mixup_data(inputs, targets,
                                               args.alpha, use_cuda)

# what do the inputs look like??? 